In [140]:
import data_processor
from importlib import reload
import pandas as pd
import shutil
import os

In [141]:
reload(data_processor)

<module 'data_processor' from '/Users/hoda_hamdy/Documents/Projects/fleetpy/FleetPy/notebooks/data_processor.py'>

In [142]:
from data_processor import *

In [143]:
scenario = 'example_pool_sc_1'
train_data_dir = f'../studies/GNN_study/results/{scenario}/train/'
save_raw_dir = f'data/{scenario}/raw/'
save_process_dir = f'data/{scenario}/processed/'
sim_duration = 7200  # unit?
sim_step = 60  # unit?

In [144]:
data_processor = DataProcessor(train_data_dir, save_raw_dir, sim_duration, sim_step)
all_data = data_processor.process_data()

100%|██████████| 120/120 [00:00<00:00, 706.56it/s]


In [145]:
shutil.rmtree(f'{save_process_dir}')
os.makedirs(f'{save_process_dir}')
for name in ['req_features', 'veh_features']:
    dfs = []
    for timestep, data in enumerate(all_data):
        df = pd.DataFrame.from_dict(data[name], orient='index')
        df['timestep'] = timestep
        dfs.append(df)
    pd.concat(dfs).to_csv(f'{save_process_dir}/{name}.csv', index=False)

In [146]:
def get_edge_attrs(info):
    edge_attrs = {}
    # TODO use more meaningful naming
    if type(info['travel_cost']) == list:
        for i in range(len(info['travel_cost'])):
            edge_attrs[f'travel_time{i}'] = info['travel_cost'][i]['travel_time']
    else:
        edge_attrs['travel_time'] = info['travel_time']
    edge_attrs['common_nghbs'] = info['common_nghbs']
    edge_attrs['jaccards_coeff'] = info['jaccards_coeff']
    edge_attrs['init_label'] = info.get('init_assign', 0)
    edge_attrs['label'] = info.get('opt_assign', 0)
    return edge_attrs

for name in ['rr_graph', 'vr_graph']:
    dfs = []
    for timestep, data in enumerate(all_data):
        df = [{'source': node, 'target': nghb, **get_edge_attrs(info)} for node, nghbs in data[name].items() for nghb, info in nghbs.items()]
        df = pd.DataFrame(df)
        df['timestep'] = timestep
        dfs.append(df)
    pd.concat(dfs).to_csv(f'{save_process_dir}/{name}.csv', index=False)